<a href="https://colab.research.google.com/github/sahupra1357/LLMAgents/blob/main/Bedrock_langchain_qa_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Amazon Bedrock
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/integrations/amazon-bedrock/langchain-qa-example.ipynb)

This workbook demonstrates how to work with Langchain [Amazon Bedrock](https://aws.amazon.com/bedrock/). Amazon Bedrock is a managed service that makes foundation models from leading AI startup and Amazon's own Titan models available through APIs.

## Install packages and import modules

In [28]:
# install packages
!python3 -m pip install -qU langchain langchain-elasticsearch langchain_community boto3 tiktoken langchain_aws langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.9 MB/s eta 0:00:00


In [30]:
# import modules
from getpass import getpass
from urllib.request import urlopen
from langchain_elasticsearch import ElasticsearchStore
#from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain_aws import BedrockEmbeddings
from langchain.llms import Bedrock
from langchain.chains import RetrievalQA
import boto3
import json

Note: boto3 is part of AWS SDK for Python and is required to use Bedrock LLM

## Init Bedrock client

To authorize in AWS service we can use `~/.aws/config` file with [configuring credentials](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html#configuring-credentials) or pass `AWS_ACCESS_KEY`, `AWS_SECRET_KEY`, `AWS_REGION` to boto3 module.

We're using second approach for our example.

In [31]:
# default_region = "us-east-1"
# AWS_ACCESS_KEY = getpass("AWS Acces key: ")
# AWS_SECRET_KEY = getpass("AWS Secret key: ")
# AWS_REGION = input(f"AWS Region [default: {default_region}]: ") or default_region

import os
from google.colab import userdata
os.environ["AWS_ACCESS_KEY"] = userdata.get('AWS_ACCESS_KEY')
os.environ["AWS_SECRET_KEY"] = userdata.get('AWS_SECRET_KEY')
os.environ["AWS_REGION"] = userdata.get('AWS_REGION')
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY')
AWS_REGION = os.getenv('AWS_REGION')

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    #service_name="bedrock",
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.


We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment. This would help create and index data easily. In the ElasticsearchStore instance, will set embedding to [BedrockEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.bedrock.BedrockEmbeddings.html) to embed the texts and elasticsearch index name that will be used in this example.

In [32]:
# # https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
# ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# # https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
# ELASTIC_API_KEY = getpass("Elastic Api Key: ")

from langchain_openai import OpenAIEmbeddings
os.environ["ELASTIC_CLOUD_ID"] = userdata.get('ELASTIC_CLOUD_ID')
os.environ["ELASTIC_API_KEY"] = userdata.get('ELASTIC_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
ELASTIC_CLOUD_ID = os.getenv('ELASTIC_CLOUD_ID')
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# bedrock_embedding = BedrockEmbeddings(client=bedrock_client,
#                                       model="amazon.titan-embed-text-v2:0")

bedrock_embedding = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="workplace_index",
    embedding=bedrock_embedding,
)

## Download the dataset

Let's download the sample dataset and deserialize the document.

In [33]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)

workplace_docs = json.loads(response.read())

## Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 500 token passages with an overlap of 0 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

metadata = []
content = []

for doc in workplace_docs:
    content.append(doc["content"])
    metadata.append(
        {
            "name": doc["name"],
            "summary": doc["summary"],
            "rolePermissions": doc["rolePermissions"],
        }
    )

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=256
)
docs = text_splitter.create_documents(content, metadatas=metadata)

## Index data into elasticsearch

Next, we will index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents). We will use Cloud ID,  Password and Index name values set in the `Create cloud deployment` step.

In [35]:
documents = vector_store.from_documents(
    docs,
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="workplace_index",
    embedding=bedrock_embedding,
)

## Init Bedrock LLM

Next, we will initialize Bedrock LLM. In the Bedrock instance, will pass `bedrock_client` and specific `model_id`: `amazon.titan-text-express-v1`, `ai21.j2-ultra-v1`, `anthropic.claude-v2`, `cohere.command-text-v14` or etc. You can see list of available base models on [Amazon Bedrock User Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)

In [46]:
#default_model_id = "amazon.titan-text-express-v1"
#default_model_id = "amazon.titan-embed-text-v2:0"
#default_model_id = "amazon.nova-lite-v1:0"
default_model_id = "meta.llama3-3-70b-instruct-v1:0"
AWS_MODEL_ID = input(f"AWS model [default: {default_model_id}]: ") or default_model_id
llm = Bedrock(client=bedrock_client, model_id=AWS_MODEL_ID)

AWS model [default: meta.llama3-3-70b-instruct-v1:0]: meta.llama3-3-70b-instruct-v1:0


## Asking a question
Now that we have the passages stored in Elasticsearch and llm is initialized, we can now ask a question to get the relevant passages.

In [47]:
retriever = vector_store.as_retriever()

qa = RetrievalQA.from_llm(llm=llm, retriever=retriever, return_source_documents=True)

questions = [
    "What is the nasa sales team?",
    "What is our work from home policy?",
    "Does the company own my personal project?",
    "What job openings do we have?",
    "How does compensation work?",
]
question = questions[1]
print(f"Question: {question}")

ans = qa({"query": question})

print("\033[92m ---- Answer ---- \033[0m")
print(ans["result"] + "\n")
print("\033[94m ---- Sources ---- \033[0m")
for doc in ans["source_documents"]:
    print("Name: " + doc.metadata["name"])
    print("Content: " + doc.page_content)
    print("-------")

Question: What is our work from home policy?
 ---- Answer ---- 
 
As of March 2020, we have a full-time work-from-home policy. However, please note that this policy has undergone changes over time. Initially, employees were allowed to work from home full-time, but starting in May 2022, a two-day in-office work requirement was introduced. Later, in May 2023, the policy was updated to require employees to work from the office three days a week, with two days designated for remote work. Employees are still expected to maintain their regular work hours, communicate effectively with their supervisors and colleagues, and adhere to the company's confidentiality and data security policies. It is essential to check with your supervisor or HR department for the most up-to-date information and to establish your specific in-office workdays.

 ---- Sources ---- 
Name: Work From Home Policy
Content: Effective: March 2020
Purpose

The purpose of this full-time work-from-home policy is to provide guid